In [3]:
import numpy as np
from transformers import TFAutoModel, BertTokenizerFast
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, TimeDistributed, Subtract, Multiply, Concatenate, Layer

In [2]:
import spacy
nlp = spacy.load("ru_core_news_lg")

In [3]:
POS_tags = {'ADJ', 'ADP', 'ADV', 'AUX',
            'CCONJ', 'DET', 'INTJ', 'NOUN',
            'NUM', 'PART', 'PRON', 'PROPN',
            'PUNCT', 'SCONJ', 'SYM', 'VERB', 
            'X', '[PAD]'}

In [1]:
DEP_tags = 'ROOT, acl, acl:relcl, advcl, advmod, amod, appos, aux, aux:pass, case, cc, ccomp, compound, conj, cop, csubj, csubj:pass, dep, det, discourse, expl, fixed, flat, flat:foreign, flat:name, iobj, list, mark, nmod, nsubj, nsubj:pass, nummod, nummod:entity, nummod:gov, obj, obl, obl:agent, orphan, parataxis, punct, xcomp'.split(', ')

In [6]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment-rurewiews')
model = TFAutoModel.from_pretrained('blanchefort/rubert-base-cased-sentiment-rurewiews', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['classifier.bias', 'classifier.weight', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [45]:
512* 768

393216

In [13]:
x = tf.constant(np.random.randint(12, size=6).reshape(3, 2))
y = tf.constant(np.random.randint(12, size=6).reshape(3, 2))

In [14]:
x

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 4,  0],
       [ 2, 10],
       [ 4,  6]])>

In [15]:
y

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[5, 4],
       [1, 7],
       [0, 5]])>

In [16]:
x = tf.concat([x, np.array([[0,0],[0,0]])],0)

In [18]:
y = tf.concat([y, np.array([[0,0],[0,0]])],0)

In [22]:
class OuterVectorSubMulConcatLayer(Layer):
    
    def __init__(self, **kwargs):
        super(OuterVectorSubMulConcatLayer, self).__init__(**kwargs)

    def call(self, inputs, masks):
        x = tf.boolean_mask(inputs[0], masks[0])
        y = tf.boolean_mask(inputs[1], masks[1])
        seq_len = x.shape[0]
        hid_size = x.shape[1]
        repeated_x = tf.reshape(tf.repeat(x, seq_len, axis=0), (seq_len, seq_len, hid_size))
        repeated_y =  tf.repeat([y], seq_len, axis=0)
        outer_subtraction = repeated_x - repeated_y
        outer_multiplication = repeated_x * repeated_y
        o = tf.reshape(tf.concat([repeated_x, repeated_y, outer_subtraction, outer_multiplication], axis=-1), 
                       (seq_len*seq_len, hid_size*4))
        return o

In [24]:
OuterVectorSubMulConcatLayer()([x[:3],y[:3]], [np.array([1,1,1,0,0])]*2)

<tf.Tensor: shape=(9, 8), dtype=int32, numpy=
array([[ 4,  0,  5,  4, -1, -4, 20,  0],
       [ 4,  0,  1,  7,  3, -7,  4,  0],
       [ 4,  0,  0,  5,  4, -5,  0,  0],
       [ 2, 10,  5,  4, -3,  6, 10, 40],
       [ 2, 10,  1,  7,  1,  3,  2, 70],
       [ 2, 10,  0,  5,  2,  5,  0, 50],
       [ 4,  6,  5,  4, -1,  2, 20, 24],
       [ 4,  6,  1,  7,  3, -1,  4, 42],
       [ 4,  6,  0,  5,  4,  1,  0, 30]])>

In [21]:
OuterVectorSubMulConcatLayer()([x,y], [np.array([1,1,1,0,0])]*2)

<tf.Tensor: shape=(9, 8), dtype=int32, numpy=
array([[ 4,  0,  5,  4, -1, -4, 20,  0],
       [ 4,  0,  1,  7,  3, -7,  4,  0],
       [ 4,  0,  0,  5,  4, -5,  0,  0],
       [ 2, 10,  5,  4, -3,  6, 10, 40],
       [ 2, 10,  1,  7,  1,  3,  2, 70],
       [ 2, 10,  0,  5,  2,  5,  0, 50],
       [ 4,  6,  5,  4, -1,  2, 20, 24],
       [ 4,  6,  1,  7,  3, -1,  4, 42],
       [ 4,  6,  0,  5,  4,  1,  0, 30]])>

In [3]:
d = 256

In [ ]:
def UDAModel(tf.keras.models.Model):
    
    def __init__(self, d, POS_tags, bert, **kwargs):
        super(UDAModel, self).__init__(**kwargs)
        self.bert = bert
        
        # POS-tag part
        self.local_pos_tgr = Dense(units = len(POS_tags),
                                   activation = 'softmax',
                                   name = 'POS-tagger')
        self.pos_clf = TimeDistributed(self.local_pos_tgr)
        
        # Syntax part
        self.local_head_repr = Dense(units = d/4,
                                     activation = 'tanh',
                                     name = 'head_repr')
        self.head_repr = TimeDistributed(self.local_head_repr)
        
        self.local_tail_repr = Dense(units = d/4,
                                     activation = 'tanh',
                                     name = 'tail_repr')
        self.tail_repr = TimeDistributed(self.local_tail_repr)
        
        o_matrix = OuterVectorSubMulConcatLayer(name = '[hh, ht, hh-ht, hh*ht]')
        
        
    def call(self, inputs):
        x = inputs[0]
        y = inputs[1]
        seq_len = x.shape[0]
        hid_size = x.shape[1]
        repeated_x = tf.reshape(tf.repeat(x, seq_len, axis=0), (seq_len, seq_len, hid_size))
        repeated_y =  tf.repeat([y], seq_len, axis=0)
        outer_subtraction = repeated_x - repeated_y
        outer_multiplication = repeated_x * repeated_y
        o = tf.concat([repeated_x, repeated_y, s, m], axis=-1)
        return o

In [ ]:
# model draft

input_ids = Input((512,))
masks = Input((512,))

lm_out = model([input_ids, masks])

local_pos_tgr = Dense(units = len(POS_tags),
                      activation = 'softmax',
                      name = 'POS-tagger')
pos_clf = TimeDistributed(local_pos_tgr)(inputs=lm_out.last_hidden_state, masks=masks)

local_head_repr = Dense(units = d/4,
                        activation = 'tanh',
                        name = 'head_repr')
head_repr = TimeDistributed(local_head_repr)(inputs=lm_out.last_hidden_state, masks=masks) # (bs, T, d/4)

local_tail_repr = Dense(units = d/4,
                        activation = 'tanh',
                        name = 'tail_repr')
tail_repr = TimeDistributed(local_tail_repr)(inputs=lm_out.last_hidden_state, masks=masks) # (bs, T, d/4)

o_matrix = OuterVectorSubMulConcatLayer(name = '[hh, ht, hh-ht, hh*ht]')([head_repr, tail_repr]) # (bs, T, T, d)